# Imports and Setup

In [1]:
!git clone https://github.com/Hari31416/transformer_from_scratch.git
!cp -r ./transformer_from_scratch/* ./
!pip install -q evaluate=='0.4.1'

Cloning into 'transformer_from_scratch'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 74 (delta 40), reused 58 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 4.28 MiB | 22.71 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [2]:
import torch
import torch.nn as nn
import evaluate

from tqdm.auto import tqdm

T = torch.Tensor
M = nn.Module


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from src.train_utils import *
from src.transformer import *

For logging to wandb. Update the API key.

In [3]:
LOG_TO_WANDB = True
if LOG_TO_WANDB:
    from kaggle_secrets import UserSecretsClient
    import wandb

    user_secrets = UserSecretsClient()
    WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")

    text = f"""machine api.wandb.ai
        login user
        password {WANDB_API_KEY}
        """
    # wandb saves credentials at /root/.netrc
    with open("/root/.netrc", "w") as f:
        f.write(text)

    wandb.init(project="Transformer_From_Scratch", name="Run 8")
else:
    wandb = None

wandb: Currently logged in as: hari31416. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240921_133322-avz5jaov
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Run 13
wandb: ⭐️ View project at https://wandb.ai/hari31416/Transformer_From_Scratch
wandb: 🚀 View run at https://wandb.ai/hari31416/Transformer_From_Scratch/runs/avz5jaov


# Configs

We start by creating the configuration for the dataset and then loading the dataset. See the notebook [lyrics](https://github.com/Hari31416/transformer_from_scratch/blob/main/notebooks/lyrics.ipynb) for a detail on the dataset creation.

In [4]:
dataset_config = GenerationDatasetConfig(
    **{
        "dataset_path": "data/songs_section_wise.json",
        "tokenizer_path": "data/tokenizer_eng_lyrics.json",
        "max_len": 80,  # amounts to 90%+of the data
        "device": device,
    }
)
dataset: GenerationDataset = dataset_config.load_object(GenerationDataset)
dataloader = dataset.get_dataloader(32, shuffle=True)

Since we want a model that generates the next word in a sequence, we will create a decoder only transformer model. Note that the same can also be achieved by using a full encoder-decoder model, however, we will use a decoder only model for simplicity and because the performance gain is not significant.

In [5]:
model = DecoderOnlyTransformer(
    target_vocab_size=dataset.tokenizer.get_vocab_size(),
    N = 6,
)
model.param_count()/1e6

27.618603

Next, we create the configuration for the trainer `TransformerTrainerForGeneration`.

In [6]:
tokenizer = dataset.tokenizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [7]:
trainer_config = TransformerTrainerForGenerationConfig(
    **{
        "model": model,
        "optimizer": optimizer,
        "criterion": bce_crit,
        "tokenizer": tokenizer,
        "max_len": dataset_config.max_len,
        "device": device,
        "wandb": wandb,
        "wandb_log_freq": 100,
        "scheduler": None,
    }
)
trainer: TransformerTrainerForGeneration = trainer_config.load_object(TransformerTrainerForGeneration)

# Training and Inference

## Training

Now, it is easy to train the model.

In [8]:
losses, train_states = trainer.train(
    data_loader=dataloader,
    num_epochs=60,
    log_freq=100,
    eval_loader=None,
)

torch.save(model.state_dict(), "trained_transformer_for_generation.pth")

  0%|          | 0/111 [00:00<?, ?it/s]

Batch Step: 1, Loss*100: 0.000413, Tokens / Sec: 384.216980, Learning Rate: 0.0001
Batch Step: 101, Loss*100: 0.000174, Tokens / Sec: 11000.087891, Learning Rate: 0.0001
Epoch: 0, Loss: 0.000002


  0%|          | 0/111 [00:00<?, ?it/s]

Batch Step: 1, Loss*100: 0.000150, Tokens / Sec: 22109.179688, Learning Rate: 0.0001
Batch Step: 101, Loss*100: 0.000128, Tokens / Sec: 10810.259766, Learning Rate: 0.0001
Epoch: 1, Loss: 0.000002


  0%|          | 0/111 [00:00<?, ?it/s]

...

Batch Step: 1, Loss*100: 0.000011, Tokens / Sec: 18870.968750, Learning Rate: 0.0001
Batch Step: 101, Loss*100: 0.000011, Tokens / Sec: 14890.936523, Learning Rate: 0.0001
Epoch: 57, Loss: 0.000000


  0%|          | 0/111 [00:00<?, ?it/s]

Batch Step: 1, Loss*100: 0.000008, Tokens / Sec: 21913.277344, Learning Rate: 0.0001
Batch Step: 101, Loss*100: 0.000011, Tokens / Sec: 14864.454102, Learning Rate: 0.0001
Epoch: 58, Loss: 0.000000


  0%|          | 0/111 [00:00<?, ?it/s]

Batch Step: 1, Loss*100: 0.000009, Tokens / Sec: 21415.460938, Learning Rate: 0.0001
Batch Step: 101, Loss*100: 0.000008, Tokens / Sec: 15222.160156, Learning Rate: 0.0001
Epoch: 59, Loss: 0.000000


## Inference

Let us see how the model performs when we generate text from a seed text. We have used the following texts:

```python
[
    "salt air and the rust on your door",
    "no other sadness in the world would do",
    "One slip and falling back into the hedge maze\nOh what a way to die",
    "It's me, hi, I'm the problem, it's me\nAt tea time, everybody agrees",
    "there i was again tonight forcing laughter faking smiles same old tired lonely place",
    "i said remember this moment in the back of my mind",
    "I was supposed to be sent away\nBut they forgot to come and get me",
    "we could leave the christmas lights up 'til january",
    "life was a willow and it bent right to your wind",
]
```

Some of these texts are in the training dataset while some are not.

In [9]:
trainer.log_sample_to_wandb(None)

{'generations': ['salt air and the rust on your door i never needed anything '
                 'more whispers of " are you sure? " " " never have i ever '
                 'before "',
                 'no other sadness in the world would do',
                 'one slip and falling back into the hedge maze oh what a way '
                 'to die',
                 "it ' s me, hi, i ' m the problem, it ' s me at tea time, "
                 'everybody agrees',
                 'there i was again tonight forcing laughter faking smiles '
                 'same old tired lonely place walls of insincerity, shifting '
                 'eyes and vacancy vanished when i saw your face all i can say '
                 'is, it was enchanting to meet you',
                 'i said remember this moment in the back of my mind the night '
                 'we stood with our shaking hands tied and i had the crowds in '
                 'stands went wild we were the kings and the queens you traded '
 

We can easily figure out which texts were in the training dataset and which were not. The last text `life was a willow and it bent right to your wind` was in the training set and the model is able to complete the text. For texts such as `I was supposed to be sent away\nBut they forgot to come and get me` were not in the training set and the model is able to complete it.